This notes is modified based on https://towardsdatascience.com/a-simple-way-to-get-a-stocks-fundamental-data-26506adf1214

In [16]:
# !pip install eod

We will use the following imports:

In [42]:
# Libraries
import pandas as pd
from eod import EodHistoricalData
from functools import reduce
from datetime import datetime, timedelta

    
# EOD Historical Data client
#Enter you api key here
client = EodHistoricalData("your api key")

In [43]:
def getFundamentals(ticker):
    """
    Returns the fundamental data from the financial data API.  Combines the quarterly balance 
    sheet, cash flow, income statement, and earnings for a specific stock ticker.
    """
    
    # Getting data
    fund_data = client.get_fundamental_equity(ticker)
    
    # Financials
    bal = pd.DataFrame(fund_data['Financials']['Balance_Sheet']['yearly']).T
    
    cf = pd.DataFrame(fund_data['Financials']['Cash_Flow']['yearly']).T
    
    inc = pd.DataFrame(fund_data['Financials']['Income_Statement']['yearly']).T
    
    # Earnings
    earn = pd.DataFrame(fund_data['Earnings']['History']).T
    
    # Merging them together
    df = reduce(
        lambda left,right: pd.merge(
            left,
            right,
            left_index=True, 
            right_index=True, 
            how='outer',
            suffixes=('', '_drop')
        ), 
        [bal, cf, inc, earn]
    )
    
    # Dropping redundant date and duplicate columns
    dup_cols = [i for i in df.columns if "date" in i or "Date" in i or "_drop" in i]
    
    df = df.drop(dup_cols, axis=1)
    
    return df

The getFundamentals() function is used to retrieve fundamental data for a specific stock ticker from a financial data API. The function takes in a single argument, ticker, which is the stock ticker symbol for the company whose data is being retrieved.

The function begins by using the client.get_fundamental_equity(ticker) method to retrieve the fundamental data for the specified ticker. This data is stored in a variable called fund_data.

Next, the function extracts the balance sheet, cash flow, and income statement data from fund_data and stores them in separate variables (bal, cf, and inc respectively). These variables are Pandas DataFrames containing the yearly financial data for the company.

The function also extracts the earnings data from fund_data and stores it in a variable called earn. This variable is also a Pandas DataFrame.

The function then uses the pd.merge() function to combine the data stored in bal, cf, inc, and earn into a single DataFrame called df. The reduce() function is used to apply the merge operation repeatedly to the four DataFrames.

Finally, the function drops any columns that contain the string "date" or "Date" in the column name and any duplicate columns by using df.drop(). It then returns the resulting DataFrame.

In [44]:
# Enter Stock code here
Stock_code = "MSFT"

msft_fundamentals = getFundamentals(Stock_code)
msft_fundamentals = msft_fundamentals.drop(columns=['currency_symbol', 'beforeAfterMarket', 'currency'])

In [45]:
pd.options.display.float_format = '{:,.2f}'.format
msft_fundamentals = msft_fundamentals.astype(float)
msft_fundamentals.index = pd.to_datetime(msft_fundamentals.index)
msft_fundamentals = msft_fundamentals.resample('Y').sum()

In [47]:
print(msft_fundamentals[-10:])

                  totalAssets  intangibleAssets  earningAssets  \
2014-12-31 172,384,000,000.00  6,981,000,000.00           0.00   
2015-12-31 176,223,000,000.00  4,835,000,000.00           0.00   
2016-12-31 193,694,000,000.00  3,733,000,000.00           0.00   
2017-12-31 241,086,000,000.00 10,106,000,000.00           0.00   
2018-12-31 258,848,000,000.00  8,053,000,000.00           0.00   
2019-12-31 286,556,000,000.00  7,750,000,000.00           0.00   
2020-12-31 301,311,000,000.00  7,038,000,000.00           0.00   
2021-12-31 333,779,000,000.00  7,800,000,000.00           0.00   
2022-12-31 364,840,000,000.00 11,298,000,000.00           0.00   
2023-12-31               0.00              0.00           0.00   

            otherCurrentAssets          totalLiab  totalStockholderEquity  \
2014-12-31    6,333,000,000.00  82,600,000,000.00       89,784,000,000.00   
2015-12-31    7,376,000,000.00  96,140,000,000.00       80,083,000,000.00   
2016-12-31    5,892,000,000.00 121,697,000